In [ ]:
import astrophot as ap
import numpy as np
import torch
from astropy.io import fits
import matplotlib.pyplot as plt

%matplotlib inline

# Modelling Point Sources

## Making a PSF model

Before we can optimize a PSF model, we need to make the model and get some starting parameters. If you already have a good guess at some starting parameters then you can just enter them yourself, however if you don't then AstroPhot provides another option; if you have an empirical PSF estimate (a stack of a few stars from the field), then you can have a PSF model initialize itself on the empirical PSF just like how other AstroPhot models can initialize themselves on target images. Let's see how that works!

In [ ]:
# First make a mock empirical PSF image
# np.random.seed(124)
psf = ap.utils.initialize.moffat_psf(2.0, 3.0, 101, 0.5)
variance = psf**2 / 100
psf += np.random.normal(scale=np.sqrt(variance))
# psf[psf < 0] = 0 #ap.utils.initialize.moffat_psf(2.0, 3.0, 101, 0.5)[psf < 0]

psf_target = ap.image.PSF_Image(
    data=psf,
    pixelscale=0.5,
)

# To ensure the PSF has a normalized flux of 1, we call
psf_target.normalize()

fig, ax = plt.subplots()
ap.plots.psf_image(fig, ax, psf_target)
ax.set_title("mock empirical PSF")
plt.show()

In [ ]:
# Now we initialize on the image
psf_model = ap.models.AstroPhot_Model(
    name="init psf",
    model_type="moffat psf model",
    target=psf_target,
)

psf_model.initialize()

# PSF model can be fit to it's own target for good initial values
# Note we provide the weight map (1/variance) since a PSF_Image can't store that information.
ap.fit.LM(psf_model, verbose=1, W=1 / variance).fit()

fig, ax = plt.subplots(1, 2, figsize=(13, 5))
ap.plots.psf_image(fig, ax[0], psf_model)
ax[0].set_title("PSF model fit to mock empirical PSF")
ap.plots.residual_image(fig, ax[1], psf_model, normalize_residuals=torch.tensor(variance))
ax[1].set_title("residuals")
plt.show()

That's pretty good! It doesn't need to be perfect, so this is already in the right ballpark, just based on the size of the main light concentration. For the examples below, we will just start with some simple given initial parameters, but for real analysis this is quite handy.

## Fitting to Multiple Stars

## Multi-Component PSF Models (`psf group model`)

Often when you really look at the data, a single ``moffat psf model`` won't quite cut it for accuracy, but it is very close. You may be able to get a better fit by combining psf models just like how you combine other models to handle an astronomical image. Here we will go over a basic example of producing a ``psf group model`` to see how it works.

In [ ]:
# First make a mock empirical PSF image
np.random.seed(124)
pixelscale = 1.0
psf1 = ap.utils.initialize.moffat_psf(1.0, 4.0, 101, pixelscale, normalize=False)
psf2 = ap.utils.initialize.moffat_psf(3.0, 2.0, 101, pixelscale, normalize=False)
psf = psf1 + 0.5 * psf2
psf /= psf.sum()
star = psf * 10  # flux of 10
variance = star / 1e5
star += np.random.normal(scale=np.sqrt(variance))

psf_target2 = ap.image.PSF_Image(
    data=star.copy() / star.sum(),  # empirical PSF from cutout
    pixelscale=pixelscale,
)
psf_target2.normalize()

point_target = ap.image.Target_Image(
    data=star,  # cutout of star
    pixelscale=pixelscale,
    variance=variance,
)

fig, ax = plt.subplots()
ap.plots.psf_image(fig, ax, psf_target2)
ax.set_title("mock empirical two component PSF")
plt.show()

In [ ]:
# Here we build a multi component PSF model

moffat_component1 = ap.models.AstroPhot_Model(
    name="psf part1",
    model_type="moffat psf model",
    target=psf_target2,
    parameters={
        "n": 1.5,
        "Rd": 4.5,
        "I0": {"value": -3.0, "locked": False},
    },
    normalize_psf=False,
)

moffat_component2 = ap.models.AstroPhot_Model(
    name="psf part2",
    model_type="moffat psf model",
    target=psf_target2,
    parameters={
        "n": 2.6,
        "Rd": 1.7,
        "I0": {"value": -2.3, "locked": False},
    },
    normalize_psf=False,
)

full_psf_model = ap.models.AstroPhot_Model(
    name="full psf",
    model_type="psf group model",
    target=psf_target2,
    models=[moffat_component1, moffat_component2],
    normalize_psf=True,
)
full_psf_model.initialize()

fig, ax = plt.subplots(1, 2, figsize=(13, 5))
ap.plots.psf_image(fig, ax[0], full_psf_model)
ax[0].set_title("PSF model initialization")
ap.plots.residual_image(fig, ax[1], full_psf_model, normalize_residuals=torch.tensor(variance))
ax[1].set_title("initial residuals")
plt.show()

Using a star cutout (or multiple cutouts like above) is the best way to fit a PSF model. In this case you are fitting a `point model` with a live PSF model. 

In [ ]:
# Here we build the point model which has a live PSF model

target = ap.image.Target_Image(
    data=psf,
    pixelscale=1.0,
    variance=variance,
)
model = ap.models.AstroPhot_Model(
    name="star",
    model_type="point model",
    target=point_target,
    psf=full_psf_model,
)
model.initialize()

In [ ]:
ap.fit.LM(model, verbose=1).fit()
print(model)
fig, ax = plt.subplots(1, 2, figsize=(13, 5))
ap.plots.psf_image(fig, ax[0], full_psf_model)
ax[0].set_title("PSF model fit to mock empirical PSF")
ap.plots.residual_image(fig, ax[1], full_psf_model, normalize_residuals=torch.tensor(variance))
ax[1].set_title("residuals")
plt.show()